In [96]:
# For 2006 to 2017
import requests
from bs4 import BeautifulSoup
music_list =[]

for year in range(2006,2018):
    url = "https://www.billboard.com/charts/year-end/"+ str(year)+"/hot-100-songs"
    response = requests.get(url)
    results_page = BeautifulSoup(response.content,'lxml')
    # print(results_page.prettify())
    article_tags = results_page.find_all('article', class_= "ye-chart-item")
    for tag in article_tags:
        artist =tag.find('div', class_= "ye-chart-item__artist").get_text().strip()
        track  = tag.find('div', class_= "ye-chart-item__title").get_text().strip()
        music_list.append((year, artist,track))
        

In [97]:
#for 2018
import requests
from bs4 import BeautifulSoup
url = "https://www.billboard.com/charts/hot-100"
response = requests.get(url)
results_page = BeautifulSoup(response.content,'lxml')
# print(results_page.prettify())
div_tags = results_page.find_all('div', class_= "chart-list-item")
for div in div_tags:
    artist= div.get("data-artist")
    track = div.get("data-title")
        

In [17]:
# !pip3 install spotipy


  Using cached https://files.pythonhosted.org/packages/59/46/3c957255c96910a8a0e2d9c25db1de51a8676ebba01d7966bedc6e748822/spotipy-2.4.4.tar.gz
  Running setup.py bdist_wheel for spotipy ... done
  Stored in directory: /Users/shazliekhan/Library/Caches/pip/wheels/76/28/19/a86ca9bb0e32dbd4a4f580870250f5aeef852870578e0427e6
Successfully built spotipy


In [ ]:
import spotipy
import numpy as np
from spotipy.oauth2 import SpotifyClientCredentials
client_credentials_manager = SpotifyClientCredentials(client_id="a31e2c1446fd4e6aa11476d8532fc939",client_secret="b52f884f9d4f464eba58daf0d5180ea4")
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
from collections import OrderedDict
def get_spotify_features(track, artist):
    #Search for Spofity song ID 
    songs=sp.search(q='track:'+track+' '+'artist:'+artist+'' , type='track')
    items = songs['tracks']['items']
#     print(items)
    if len(items) ==0:
        return([0]*18)
    else:
        track = items[0]
        song_id = str(track["id"])
        #Use ID to get Song features
        track_features=sp.audio_features(song_id)
        if len(track_features[0]) <18:
            return([0]*18)
        else:
            features = np.array(track_features)[0]
            #Order Dictionary
#             print(len(track_features[0]))
            features = OrderedDict(features)
            return (features)
            #Get Dictionary values
            feature_values = features.values()
            return(feature_values)

# get_spotify_features("Sicko Mode", "Travis Scott")    
audio_features = []
for i in range(len(music_list)-1):
    year = music_list[i][0]
    track = music_list[i][2]
    artist= music_list[i][1]
    get_spotify_features(track,artist)
    danceability, energy, key, loudness, mode,speechiness,acousticness, instrumentalness, liveness, valence, tempo, type_, id_,uri, track_href,analysis_url,duration_ms, time_signature = get_spotify_features(track, artist)
    audio_features.append((year,artist,track,danceability, energy, key, loudness, mode, speechiness,acousticness, instrumentalness, liveness, valence, tempo, type_,duration_ms, time_signature))


In [ ]:
import pandas as pd
labels = ["Year", "Artist", "Track", "Danceability","Energy", "Key", "Loudness", "Mode", "Speechiness", "Acousticness",
          "Instrumentalness", "Liveness", "Valence", "Tempo","Type_","Duration_ms", "Time_Signature"]
df =pd.DataFrame(audio_features, columns=labels)
df.tail()